In [6]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=MUxW0VpiTm0BbqueXzEzTTs3rcA4Q3rIS0qllKACIkc&tc=48wOB3Cu8q21p_oDtZeMn7CpWnjKRCin3Pi8g_qwb30&cc=zx_OxlatGHSE5bjbgK2kB6AAUk0dCFbBa0lOGP_fpFo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPVJtTEge-8DcsaSs3g24ICNN-vSD8BTArpeaRwLT8VqDhku9OxezY

Successfully saved authorization token.


In [1]:
!pip install jdatetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import datetime
from dateutil.relativedelta import relativedelta
import jdatetime

def jalali_to_gregorian(date):
    date_gregorian = jdatetime.date(year=date.year, month=date.month, day=date.day).togregorian()
    return date_gregorian.strftime("%Y-%m-%d")

# Define the AOI geometry
AOI = ee.Geometry.Polygon(
  [[[50.01184816666665, 36.83352820000001],
  [50.01184816666665, 37.01750912500001],
  [50.24125524999999, 37.01750912500001],
  [50.24125524999999, 36.83352820000001],
  [50.01184816666665, 36.83352820000001]]])



# Convert the end date to Gregorian
end_date_jalali = "1401-02-13"  # Replace with your Jalali end date
end_date = jdatetime.datetime.strptime(end_date_jalali, '%Y-%m-%d').date()
END_DATE = jalali_to_gregorian(end_date)

# Calculate the start date by subtracting 14 days from the end date
start_date = end_date - jdatetime.timedelta(days=14)

START_DATE = jalali_to_gregorian(start_date)

# Set the remaining parameters
CLOUD_FILTER = 30
CLD_PRB_THRESH = 70
NIR_DRK_THRESH = 0.25
CLD_PRJ_DIST = 2
BUFFER =50
END_DATE, START_DATE

('2022-05-03', '2022-04-19')

In [8]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [9]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [10]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [11]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudsdwmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [12]:

def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()
    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    cloudshdwmask = img.select('cloudsdwmask').selfMask()
    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map

    m.add_ee_layer(img,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 image', True, 1, 9)

    m.add_ee_layer(cloudshdwmask, {'palette': 'orange'},
                   'cloudshadowmask', True, 0.5, 9)
    m.add_ee_layer(clouds, {'palette': 'orange'},
                   'clouds', True, 0.5, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [13]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [14]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [15]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
#get_band_names(s2_sr_cld_col_eval_disp)

In [16]:
display_cloud_layers(s2_sr_cld_col_eval_disp)

In [20]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudsdwmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.updateMask(not_cld_shdw)

In [21]:
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [22]:
def get_image_count(collection):
    # Get the number of images in the collection
    image_count = collection.size()

    # Return the image count as an integer
    return int(image_count.getInfo())

In [23]:
# Get the number of images in the collection
image_count = get_image_count(s2_sr_cld_col_eval)

# Print the number of images
print("Image Count:", image_count)

Image Count: 4


In [24]:
def get_band_names(collection):
    # Get the first image in the collection
    first_image = ee.Image(collection.first())

    # Get the band names of the first image
    band_names = first_image.bandNames()

    # Return the band names as a list
    return band_names.getInfo()

In [ ]:
get_band_names(s2_sr_cld_col)

In [39]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

s2_sr_first = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask).sort("CLOUDY_PIXEL_PERCENTAGE")
                             .first())

In [40]:
# Check available band names in s2_sr_median
available_bands = s2_sr_median.bandNames().getInfo()
print("Available bands:", available_bands)


Available bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'probability', 'clouds', 'dark_pixels', 'cloud_transform', 'shadows', 'cloudsdwmask']


In [55]:
def display_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()
    img_col_nocldshdw = (col.map(add_cld_shdw_mask)
                           .map(apply_cld_shdw_mask).mosaic())
    s2_sr_image = col.sort("CLOUDY_PIXEL_PERCENTAGE").first()

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    #ADD layers
    # Loop through each image in the collection.
    images = col.toList(col.size())
    for i in range(col.size().getInfo()):
        image = ee.Image(images.get(i))

        # Add the image as a layer to the folium map.
        m.add_ee_layer(image,
                       {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                       f'Image {i+1}', True, 1, 9)

    m.add_ee_layer(img,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 image mosaic', True, 1, 9)
    m.add_ee_layer(s2_sr_image,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'the least cloud of collection', True, 1, 9)
    m.add_ee_layer(img_col_nocldshdw,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'mosaic_nocldshdw', True, 1, 9)

    m.add_ee_layer(s2_sr_first,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud first', True, 1, 9)
    m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud median', True, 1, 9)


    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [56]:
display_layers(s2_sr_cld_col)

In [ ]:


# Export each band individually
# Specify the bands to export
bandsToExport = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']
for bandName in bandsToExport:
        # Select the band
        band = s2_sr_median.select(bandName)

        # Specify the export parameters
        exportParams = {
            'image': band,
            'description': 'sentinel2_R64C26_ai_' + bandName,
            'folder': 'Sentinel2_Images',  # Specify the folder in Google Drive to save the image
            'scale': 10,  # Specify the desired spatial resolution (e.g., 10 meters)
            'crs': 'EPSG:4326',  # Specify the coordinate reference system
            'region': AOI  # Specify the region of interest
        }

        # Start the export task
        task = ee.batch.Export.image.toDrive(**exportParams)
        task.start()

        print('Exporting ' + bandName + ' band to Google Drive. Please wait...')

Exporting B2 band to Google Drive. Please wait...
Exporting B3 band to Google Drive. Please wait...
Exporting B4 band to Google Drive. Please wait...
Exporting B5 band to Google Drive. Please wait...
Exporting B6 band to Google Drive. Please wait...
Exporting B7 band to Google Drive. Please wait...
Exporting B8 band to Google Drive. Please wait...
Exporting B8A band to Google Drive. Please wait...
Exporting B11 band to Google Drive. Please wait...
Exporting B12 band to Google Drive. Please wait...
Exporting TCI_R band to Google Drive. Please wait...
Exporting TCI_G band to Google Drive. Please wait...
Exporting TCI_B band to Google Drive. Please wait...


In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'sentinel2_R64C26_ai_TCI_B',
 'creation_timestamp_ms': 1687174741731,
 'update_timestamp_ms': 1687174986141,
 'start_timestamp_ms': 1687174867451,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1w4GCzYGx7TqqUWOpuODuW0W7vnymD6th'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 48.546600341796875,
 'id': 'JKFDIGFYYIKNCYPZDDX5TXJK',
 'name': 'projects/earthengine-legacy/operations/JKFDIGFYYIKNCYPZDDX5TXJK'}